## Imports

In [0]:
from pyspark.sql.functions import *

## Loan purpose risk analysis

In [0]:
gold_loan_risk = spark.table('msme_risk_analytics.silver_loan_default') \
    .groupBy('loan_purpose', 'Region') \
    .agg(
        count('*').alias('total_loans'),
        sum('Status').alias('total_defaults'),
        (sum('Status')/count('*')*100).alias('default_rate'),
        avg('loan_amount').alias('avg_loan_amount'),
        avg('Credit_Score').alias('avg_credit_score')
    )

gold_loan_risk.write.format('delta').mode('overwrite') \
    .saveAsTable('msme_risk_analytics.gold_loan_risk_by_purpose')

print(f"✓ Created: gold_loan_risk_by_purpose ({gold_loan_risk.count()} rows)")

✓ Created: gold_loan_risk_by_purpose (19 rows)


##  Credit score bands

In [0]:
gold_credit_bands = spark.table('msme_risk_analytics.silver_ml_features') \
    .groupBy('credit_category') \
    .agg(
        count('*').alias('total_loans'),
        sum('Status').alias('defaults'),
        (sum('Status')/count('*')*100).alias('default_rate'),
        avg('risk_score').alias('avg_risk_score')
    )

gold_credit_bands.write.format('delta').mode('overwrite') \
    .saveAsTable('msme_risk_analytics.gold_credit_risk_bands')

print(f"✓ Created: gold_credit_risk_bands")
gold_credit_bands.show()

✓ Created: gold_credit_risk_bands
+---------------+-----------+--------+------------------+------------------+
|credit_category|total_loans|defaults|      default_rate|    avg_risk_score|
+---------------+-----------+--------+------------------+------------------+
|           Fair|       4147|    1482| 35.73667711598746| 54.26520824797837|
|           Good|       4032|    1394|34.573412698412696| 50.11911675220894|
|      Excellent|       6258|    2269| 36.25759028443592|45.034783604949645|
|           Poor|       2115|     750| 35.46099290780142| 57.18681688367446|
+---------------+-----------+--------+------------------+------------------+

